# Post analysis script

First created: 2018-08-06    
By: Hirotaka Iwaki     


This is to organize the meta-analysis results and provide some information for interpretations.    
Trying to target two sets of variants.    
1. The set of variants exceed the 5E-8 threshold in any of the meta-analysis.
2. The 92 recently identified PD risk variants.

### Extracting interesting variants and get p-value for each outcomes
1. Get the list of significant 5E-8 variants across outcomes
2. Add newly identified 92 variants (Meta5)
3. Add 31 variants in used in a recent PD-progression analysis.
4. Take p-values of the variants from the analysis for each outcome
5. Create cross-tab of $-log_{10}P$ for  [$variants \times outcomes$]

(*The next cell should be uncommented  before starting the following*)

In [49]:
# %%bash
# rm -rf post
# mkdir -p post
# rm -rf /data/LNG/Hirotaka/progGWAS/meta/rvtest/MMSE #MMSE_baseline is deleted.

In [ ]:
%%bash
# step1: Get the list of variants
for ANALYSIS in "surv" "rvtest";do
    mkdir post/$ANALYSIS
    RES_FOLDER="/data/LNG/Hirotaka/progGWAS/meta/$ANALYSIS"
    for OUTCOME in $(ls $RES_FOLDER);do
        awk '$10 < 5e-8 {print $1}' $RES_FOLDER/$OUTCOME/meta2.tbl > post/$ANALYSIS/sigV_"$OUTCOME".list
    done
done
rm -f post/sigV.list
cat post/*/sigV*.list | sort -u  >> post/sigV.list
#  rm post/sigV_*.list
echo 'complete step1'

# step2,3: Get the list of variants from Meta5, PD-progressio analysis 
tail -n +2 data/Meta5.tab | cut -f1 > post/Meta5.list
awk 'BEGIN{FS=","}/NC_/{print "chr"$5":"$6}' data/PriorSNPsGRCh37.csv > post/snp31.list
cat post/Meta5.list post/sigV.list post/snp31.list | uniq > post/allV.list
echo "complete step2,3"

# step4: Get P of the variants from each result (regardless of significant or not)
for ANALYSIS in surv rvtest;do
    RES_FOLDER=/data/LNG/Hirotaka/progGWAS/meta/$ANALYSIS
    for OUTCOME in $(ls $RES_FOLDER);do
       grep -f post/allV.list $RES_FOLDER/$OUTCOME/meta2.tbl |\
            cut -f 1,10 > post/$ANALYSIS/allV_"$OUTCOME".tbl
    done
done
echo 'complete step4'

# step5: P Cross-tabulated over outcomes
## R program detect folders in "post" as a analysis
echo '
library(dplyr);library(data.table)
ANALYSES = list.dirs("./post/", full.names=F)[-1] # delete self
for (ANALYSIS in ANALYSES){
    ALLFILES = list.files(paste("post/", ANALYSIS, "/", sep=""))
    FILES=ALLFILES[grepl("allV_",ALLFILES)]
    OUTCOMES = substring(FILES, 6, nchar(FILES)-4)
    for(i in 1:length(OUTCOMES)){
        dt_i = try(fread(paste("post",ANALYSIS,FILES[i],sep="/"), header=F), silent=T)
        if(class(dt_i)[1]=="try-error"){next}
        names(dt_i) = c("V1",paste(OUTCOMES[i], ANALYSIS, sep="_"))
        if(!exists("dt1")){dt1 = dt_i}else{    dt1 = full_join(dt1, dt_i, by = "V1")}
    }
    cat(paste("complete step5 -", ANALYSIS, "\n"))
}
dt2 = dt1 %>% mutate_at(vars(names(dt1)[2:ncol(dt1)]), funs(-log10(.)))
write.table(dt2, paste("post/allV.tab", sep=""), row.names = F, quote = F, sep = "\t")
' > post/_CrossTab.R
module load R
Rscript --vanilla post/_CrossTab.R

### Combining the annotation information

1. get rsID from variants' location and reference file.
2. get informaion from dbSNP using rsID.
3. join the information to the original cross-tab and sort by position

In [50]:
%%bash
# Step1:  get rsIDs for the variants
sed 's/^chr//g' post/allV.tab | (head -n 1 - && tail -n +2 - | LANG=C sort) |\
    LANG=C join -t$'\t' --header - ../tools/rs_37_sort.txt > post/allV_RS.tab

In [10]:
# Preparation
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re
import time
import pandas as pd

In [ ]:
# Step2: pull information from dbSNP
data = pd.read_table("post/allV_RS.tab")
IDs = data["ID"] # dbSNP ID

SYMBOLs = []
SOTERMs = []
FXNCLASSs=[]
for ID in IDs:
    print(ID)
    response = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=' + ID[2:] + '&report=XML') 
    html_str = response.text 
    bs = BeautifulSoup(html_str, "html5lib") 
    try:
        SYMBOL = bs.fxnset['symbol'] 
        SOTERM = bs.fxnset['soterm']
        FXNCLASS = bs.fxnset['fxnclass']
    except (TypeError, KeyError):
        if len(bs.find_all('fxnset'))>1:
            print("search from the second tag")
            SYMBOL = bs.find_all('fxnset')[1]['symbol'] 
            SOTERM = bs.find_all('fxnset')[1]['soterm']
            FXNCLASS = bs.find_all('fxnset')[1]['fxnclass']
        else:
            SYMBOL = "NA" 
            SOTERM = "NA"
            FXNCLASS = "NA"
    SYMBOLs.append(SYMBOL)
    SOTERMs.append(SOTERM)
    FXNCLASSs.append(FXNCLASS)
    print (SYMBOL, SOTERM, FXNCLASS)
    time.sleep(1/3) # three requests per second (Guideline)

In [30]:
#Step3 join the cross-tab with the new information
data2= data.assign(SYMBOL=SYMBOLs, SOTERM=SOTERMs, FXNCLASS=FXNCLASSs)
## ordering the 
forIdx = data2["V1"].str.split(":", expand=True).applymap(int).sort_values(by=[0, 1])
df = data2.reindex(forIdx.index)
df.head()
df.to_csv("outputs/allV_info.csv", sep='\t')

In [129]:
df=pd.read_csv("outputs/allV_info.csv", sep='\t', index_col=0, keep_default_na=False, na_values="")
df.shape

(823, 38)

In [12]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_out = df.iloc[:, 1:(df.shape[1]-6)]
df_out.index = df['V1'] + "[" + df['FXNCLASS'] + "] " + df['SYMBOL']
fig=plt.figure(figsize=(15, 80), dpi= 80, facecolor='w', edgecolor='k')
sns.heatmap(df_out,cmap='RdBu_r', vmin=0, center=1.301, vmax=7.301)

When I look at the MMESE_slope, it has many hits but they don't overlap with other relevant outcomes (e.g. DEMENTIA_surv). I used a new algorithm for a slope analysis (data projection to a complementary orthogonal matirx) so I need to double check it with glmm analysis. (It would take a few days).    
To get an idea from more confirmative results, ** I will remove variants only significant in the slope outcomes. **

In [131]:
df_wo_slope = df.loc[:,[i for i in list(df) if not "slope" in i]]
df_wo_slope_sig = (df_wo_slope.iloc[:, 1:(df_wo_slope.shape[1]-6)] > 7.301).sum(axis=1)
df_wo_slop_less = df_wo_slope.loc[df_wo_slope_sig>0]

In [ ]:
dft = df_wo_slop_less
dft_out = dft.iloc[:, 1:(dft.shape[1]-6)]
dft_out.index = dft['V1'] + " [" + dft['FXNCLASS'] + "] " + dft['SYMBOL']
dft_out.head()
fig=plt.figure(figsize=(10,60), dpi= 80, facecolor='w', edgecolor='k')
sns.heatmap(dft_out,cmap='RdBu_r', vmin=0, center=1.301, vmax=7.301)